In [28]:
# %pip install pymongo
pip install dotenv

In [29]:
import os
from pymongo import MongoClient
from openai import OpenAI
import json
from dotenv import load_env

In [30]:

client = MongoClient("mongodb://localhost:27017")
db = client["my_database"]
#Delete all documents in the collection before we start 
collection = db['opportunities_glassdoor']
open_ai_api_key="d"

#retievie 1 docuemnt from the collection
doc = collection.find({})
# print(doc["data"])
data = doc["data"]

tools = [
  {
    "type": "function",
    "function": {
      "name": "get_job_json",
      "description": "Turn this raw text containing Job description into Structions JSON with the keys:(title,company,location,salary,description,)",
      "parameters": {
        "type": "object",
        "properties": {
          "title": {
            "type": "string",
            "description": "The name of the job position, e.g. Junior Software Engineer",
          },
          "company": {"type": "string", "description": "The name of the company, e.g. Google"},
          "season": {"type": "string", "description": "The relevant time in seasons, e.g. Summer / 2024"},
          "location": {"type": "string","description": "The city and province, e.g. Ottawa, Ontario"},
          "description": {"type": "string","description": """A description of the job and any other information about the job, e.g. 
                          Who are we?
Our mission is to scale intelligence to serve humanity. We’re training and deploying frontier models for developers and enterprises who are building AI systems to power magical experiences like semantic search, RAG, and agents. We believe that our work is instrumental to the widespread adoption of AI.

We obsess over what we build. Each one of us is responsible for contributing to increasing the capabilities of our models and the value they drive for our customers. We like to work hard and move fast to do what’s best for our customers.

Cohere is a team of researchers, engineers, designers, and more, who are passionate about their craft. Each person is the one of the best in the world at what they do. We believe that a diverse range of perspectives is a requirement for building great products.

Join us on our mission and shape the future!

Why this role?
This role is for students who are excited about building the next generation of machine learning models and NLP products. Our SWE roles can cover creating datasets for machine learning, scaling the pods to serve our API, or even building out new security features on our platform. We don't distinguish much between interns and full-time employees, and you’ll have plenty of opportunities to push code to production. You'll have full autonomy and ownership over high-impact work, and will be backed by the support of an incredible team or leaders & mentors. Join us at a pivotal moment, shape what we build, and wear multiple hats!

We're currently hiring for multiple teams and roles, including Frontend, Backend, Full-stack, and Infrastructure roles. We'll take your interests & experience into account throughout the application process.

Our recruitment process will begin in the upcoming weeks, and we will be carefully reviewing applications and assessing potential candidates for our internships. Should we find a suitable match with your qualifications and our requirements, we will be in touch to discuss the opportunity further and to advance your application to the next stage. Please apply by June 28th.

Please Note: To be eligible for this position, you should be currently enrolled in a post-secondary program and available for a full-time 3-6 month internship, co-op, or research work term. We have offices in Toronto, San Francisco, New York, and London but embrace being remote-friendly! There are no restrictions on where you can be located for this role.
As a Software Engineering Intern, you will:
Ship delightful experiences for our user-facing products, meticulously crafting code for browsers or server code.
Build features for the API platform that directly impact users.
Design and implement robust data pipelines (crawlers, storage, filters).
Design and implement scalable services or infrastructure for machine learning development.
Build internal tooling (CI/CD, dev utilities) to move faster together.
Build tech writing skills through maintaining and contributing to technical documentation, both internal and external facing.
Keep up with the cutting edge and adopt new technologies to improve performance and reliability across Cohere.

If some of the above doesn’t line up perfectly with your experience, we still encourage you to apply! If you consider yourself a thoughtful worker, a lifelong learner, and a kind and playful team member, Cohere is the place for you.

We value and celebrate diversity and strive to create an inclusive work environment for all. We welcome applicants of all kinds and are committed to providing an equal opportunity process. Cohere provides accessibility accommodations during the recruitment process. Should you require any accommodation, please let us know and we will work with you to meet your needs.

Our Perks:
An open and inclusive culture and work environment
‍ Work closely with a team on the cutting edge of AI research
Free daily lunch
Full health and dental benefits, including a separate budget to take care of your mental health
Personal enrichment benefits towards arts and culture, fitness and well-being
Remote-flexible, offices in Toronto, New York, San Francisco and London and coworking stipends
✈️ Paid vacation
Show less
Company overview
Size
1 to 50 Employees
Founded
--
Type
Company - Private
Industry
Advertising & Public Relations
Sector
Media and communication
Revenue"""},
        },
        "required": ["location","company","title","season","description",],
      },
    }
  }
]


def chat_completion_request(messages, tools=None, tool_choice=None, model="gpt-3.5-turbo-0125"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


# CALL CHAT COMPLETION TO TEST
messages = []
messages.append({"role": "system", "content": "If information is not avalible do not make asumtions, simply return 'NA' ."})
messages.append({"role": "user", "content": f"{data}"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
print (assistant_message)

if assistant_message.tool_calls:
    tool_call = assistant_message.tool_calls[0]
    json_content = tool_call.function.arguments  # Access the arguments directly
    parsed_json = json.loads(json_content)
    print(parsed_json)
else:
    print("No tool calls found in the assistant message.")

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_SwpUxQdCMSCAIHu0AOYrjnfM', function=Function(arguments='{"title":"Frontend Software Engineer Intern","company":"GreatFrontEnd","season":"NA","location":"NA","description":"This is a paid, fully remote, internship role open to international applicants. As a Front End Engineering intern, you will partner closely with GreatFrontEnd\'s engineering, product and design teams to conceptualize, scope and build new projects from the ground up. You will work in an environment with a heavy emphasis on highly-polished user experiences, experimentation, and front end infrastructure. In doing so, you will grow rapidly in your front end skills, while building projects and tools that millions of front end engineers will potentially use."}', name='get_current_weather'), type='function')])
{'title': 'Frontend Software Engineer Intern', 'company': 'GreatFrontEnd', 'season': 'NA', '

In [31]:
data

"GreatFrontEnd\nFrontend Software Engineer Intern (CA, Remote)\nRemote\nEasy Apply\nEasy Apply\nWhat you'll do\nThis is a paid, fully remote, internship role open to international applicants.\nAs a Front End Engineering intern, you will partner closely with GreatFrontEnd's engineering, product and design teams to conceptualize, scope and build new projects from the ground up. You will work in an environment with a heavy emphasis on highly-polished user experiences, experimentation, and front end infrastructure. In doing so, you will grow rapidly in your front end skills, while building projects and tools that millions of front end engineers will potentially use.\nResponsibilities\nWork with a team of technically sophisticated engineers who put users first to craft new products and features that impact hundreds of thousands of engineers around the world\nPartner with business, product and design to translate growth strategies into technical plans.\nScope, design, and develop new technic